# Título  Feature Engineering
## Autor: Jose Chelquer
## Fecha de última modificación: 23/10/2025
## Descripción:

Agrega features

## Parámetros

< Descripción de cada uno de los parámetros que utiliza el job >


In [ ]:
usar_gdrive=True    # Poner en true si se va a ejecutar con archivos de google drive en máquinas locales
#usar_gdrive=False

In [ ]:
# Columnas a sumar
a_sumar_eliminar_originales=True
a_sumar = [
    [['ctarjeta_visa', 'ctarjeta_master'], 'ctarjeta'],
    [['ctarjeta_visa_transacciones', 'ctarjeta_master_transacciones'], 'ctarjeta_transacciones'],
    [['mtarjeta_visa_consumo', 'mtarjeta_master_consumo'], 'mtarjeta_consumo'],
    [['cprestamos_personales', 'cprestamos_prendarios', 'cprestamos_hipotecarios'], 'cprestamos
    [['mprestamos_personales', 'mprestamos_prendarios', 'mprestamos_hipotecarios'], 'mprestamos'],
    [['cplazo_fijo', 'cinversion1', 'cinversion2'], 'cinversion']
    [['mplazo_fijo_pesos', 'minversion1_pesos', 'minversion2'], 'minversion_pesos']
    [['mplazo_fijo_dolares', 'minversion1_dolares'], 'minversion_dolares']
    [['cseguro_vida', 'cseguro_auto', 'cseguro_vivienda', 'cseguro_accidentes_personales'], 'cseguro']
    [['ctarjeta_visa_debitos_automaticos', 'ctarjeta_master_debitos_automaticos'], 'ctarjeta_debitos_automaticos']
    [['mttarjeta_visa_debitos_automaticos', 'mttarjeta_master_debitos_automaticos'], 'mtarjeta_debitos_automaticos']
    [['cpagodeservicios', 'cpagomiscuentas'], 'cpagos']
    [['mpagodeservicios', 'mpagomiscuentas'], 'mpagos']
    [['Master_msaldopesos', 'Visa_msaldopesos'], 'Tarjeta_msaldopesos']
    [['Master_msaldodolares', 'Visa_msaldodolares'], 'Tarjeta_msaldodolares']
    [['Master_msaldototal', 'Visa_msaldototal'], 'Tarjeta_msaldototal']
    [['Master_mconsumospesos', 'Visa_mconsumospesos'], 'Tarjeta_mconsumospesos']
    [['Master_mconsumosdolares', 'Visa_mconsumosdolares'], 'Tarjeta_mconsumosdolares']
    [['Master_mlimitecompra', 'Visa_mlimitecompra'], 'Tarjeta_mlimitecompra']
    [['Master_madelantopesos', 'Visa_madelantopesos'], 'Tarjeta_madelantopesos']
    [['Master_madelantodolares', 'Visa_madelantodolares'], 'Tarjeta_madelantodolares']
    [['Master_mpagado', 'Visa_mpagado'], 'Tarjeta_mpagado']
    [['Master_mpagospesos', 'Visa_mpagospesos'], 'Tarjeta_mpagospesos']
    [['Master_mpagosdolares', 'Visa_mpagosdolares'], 'Tarjeta_mpagosdolares']
    [['Master_mconsumototal', 'Visa_mconsumototal'], 'Tarjeta_mconsumototal']
    [['Master_cadelantosefectivo', 'Visa_cadelantosefectivo'], 'Tarjeta_cadelantosefectivo']
    [['Master_mpagominimo', 'Visa_mpagominimo'], 'Tarjeta_mpagominimo']
    ]




## Input

< Archivos de datos (csv.gz) con sus paths que van a consumirse por el job>

In [ ]:
dataset_path='/content/drive/MyDrive/Data Science y similares/Maestría Data Mining Exactas/dmeyf/dmeyf2024/datasets/'
#dataset_path = '~/buckets/b1/datasets'
dataset_file='competencia_01.csv'
#dataset_file = 'competencia_02_crudo.csv.gz'

## Output

< Archivos, bases de datos, modelos que va a generar el job>

In [ ]:
output_file='competencia_02_ampliada.csv.gz'
duckdb_file='competencia_02.db'

## Procesos

### Paquetes necesarios

In [ ]:
!pip install duckdb

## Código del proceso

< Todo el código a partir de aquí debe poder ejecutarse sin necesidad de parametrizar nada>

In [ ]:
if usar_gdrive:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install duckdb
%pip install jupysql
%pip install duckdb-engine

In [ ]:
import duckdb
import pandas as pd



In [ ]:
file_path = os.path.join(dataset_path, dataset_file)
# Conectar a una base de datos en memoria o en un archivo
duckdb_path=os.path.join(dataset_path, duckdb_file)

conn = duckdb.connect(duckdb_path)


In [ ]:
def do_sql(sql):
  print (sql)
  resultado=conn.execute(sql).fetchall()
  print (resultado)

In [ ]:
# Leer el CSV y cargarlo en una tabla de DuckDB
sql=f"DROP TABLE IF EXISTS competencia_02"
conn.execute(sql)
sql = f"CREATE TABLE competencia_02 AS SELECT * FROM read_csv_auto('{file_path}', COMPRESSION='GZIP')"
print (sql)
conn.execute(sql)


In [ ]:
do_sql('CREATE OR REPLACE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0);')



,Success


In [ ]:

%%sql --no-exeute
create or replace table competencia_01 as
select *
    ,suma_sin_null(ctarjeta_visa, ctarjeta_master) as ctarjeta
    ,suma_sin_null(ctarjeta_visa_transacciones, ctarjeta_master_transacciones) as ctarjeta_transacciones
    ,suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo) as mtarjeta_consumo
    ,suma_sin_null(cprestamos_personales, suma_sin_null(cprestamos_prendarios, cprestamos_hipotecarios)) as cprestamos
    ,suma_sin_null(mprestamos_personales, suma_sin_null(mprestamos_prendarios, mprestamos_hipotecarios)) as mprestamos
    ,suma_sin_null(cplazo_fijo, suma_sin_null(cinversion1, cinversion2)) as cinversion
    ,suma_sin_null(mplazo_fijo_pesos, suma_sin_null(minversion1_pesos, minversion2)) as minversion_pesos
    ,suma_sin_null(mplazo_fijo_dolares, minversion1_dolares) as minversion_dolares
    ,suma_sin_null(cseguro_vida, suma_sin_null(cseguro_auto, suma_sin_null(cseguro_vivienda, cseguro_accidentes_personales))) as cseguro
    ,suma_sin_null(ctarjeta_visa_debitos_automaticos, ctarjeta_master_debitos_automaticos) as ctarjeta_debitos_automaticos
    ,suma_sin_null(mttarjeta_visa_debitos_automaticos, mttarjeta_master_debitos_automaticos) as mtarjeta_debitos_automaticos
    ,suma_sin_null(cpagodeservicios, cpagomiscuentas) as cpagos
    ,suma_sin_null(mpagodeservicios, mpagomiscuentas) as mpagos
    ,suma_sin_null(Master_msaldopesos, Visa_msaldopesos) as Tarjeta_msaldopesos
    ,suma_sin_null(Master_msaldodolares, Visa_msaldodolares) as Tarjeta_msaldodolares
    ,suma_sin_null(Master_msaldototal, Visa_msaldototal) as Tarjeta_msaldototal
    ,suma_sin_null(Master_mconsumospesos, Visa_mconsumospesos) as Tarjeta_mconsumospesos
    ,suma_sin_null(Master_mconsumosdolares, Visa_mconsumosdolares) as Tarjeta_mconsumosdolares
    ,suma_sin_null(Master_mlimitecompra, Visa_mlimitecompra) as Tarjeta_mlimitecompra
    ,suma_sin_null(Master_madelantopesos, Visa_madelantopesos) as Tarjeta_madelantopesos
    ,suma_sin_null(Master_madelantodolares, Visa_madelantodolares) as Tarjeta_madelantodolares
    ,suma_sin_null(Master_mpagado, Visa_mpagado) as Tarjeta_mpagado
    ,suma_sin_null(Master_mpagospesos,Visa_mpagospesos) as Tarjeta_mpagospesos
    ,suma_sin_null(Master_mpagosdolares, Visa_mpagosdolares) as Tarjeta_mpagosdolares
    ,suma_sin_null(Master_mconsumototal, Visa_mconsumototal) as Tarjeta_mconsumototal
    ,suma_sin_null(Master_cadelantosefectivo, Visa_cadelantosefectivo) as Tarjeta_cadelantosefectivo
    ,suma_sin_null(Master_mpagominimo, Visa_mpagominimo) as Tarjeta_mpagominimo




from competencia_01

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [ ]:
#
%%sql --no-execute
create or replace table competencia_01 as
select *
    ,suma_sin_null(mcuentas_saldo, minversion_pesos) as msaldos_e_inversion
from competencia_01

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [ ]:
%%sql
SELECT column_name FROM information_schema.columns WHERE table_name = 'competencia_01' and column_name like '%decil%'


,column_name


In [ ]:
# Exportar los nombres de columnas
#
# Ejecuta la consulta SQL para obtener los nombres de las columnas desde el esquema de información
df = %sql SELECT column_name FROM information_schema.columns WHERE table_name = 'competencia_01';

# Convierte el resultado en un DataFrame de pandas
column_df = pd.DataFrame(df)
# Escribe los nombres de las columnas en un archivo CSV
column_df.to_csv(dataset_path+'/nombres_columnas_competencia_01.csv', index=False)

TAREA: Escriba una macro para hacer un ratio de dos variables que sea seguro, donde no solo hay campos con null, también esta el problema de la división por cero. Como es costumbre comparta su solución por este canal. Lea https://duckdb.org/docs/sql/functions/numeric.html para referencias de funciones que puede usar.

---

"Claro!" me dirá, mientras lee esto con un mate en la mano, "para cosas fáciles usar SQL alcanza, pero para algo más complicado como crear campos contra el data drifting es difícil".... elija su medicina:

In [ ]:
%%sql
CREATE OR REPLACE MACRO division_segura (a, b) AS (
  CASE
    WHEN b IS NULL THEN 0
    WHEN a IS NULL THEN 0
    WHEN b = 0 THEN 0
    WHEN a = 0 THEN 0
    ELSE a / b
  END
  )



,Success


In [ ]:
# grabar competencia_01_aumentada
def grabar_aumentada():
  %sql   COPY competencia_01 TO '/content/drive/MyDrive/Data Science y similares/Maestría Data Mining Exactas/dmeyf/dmeyf2024/datasets/competencia_01_aumentada.csv' (FORMAT CSV, HEADER TRUE);
  print ("Se grabó /Data Science y similares/Maestría Data Mining Exactas/dmeyf/dmeyf2024/datasets/competencia_01_aumentada.csv")
# grabar competencia_01_reducida
def grabar_reducida():
   %sql   COPY competencia_01 TO '/content/drive/MyDrive/Data Science y similares/Maestría Data Mining Exactas/dmeyf/dmeyf2024/datasets/competencia_01_reducida.csv' (FORMAT CSV, HEADER TRUE);
   print ("Se grabó /Data Science y similares/Maestría Data Mining Exactas/dmeyf/dmeyf2024/datasets/competencia_01_reducida.csv")

In [ ]:
%%sql --no-execute
create or replace table competencia_01 as
select *
    ,division_segura(mcomisiones, msaldos_e_inversion) as ratio_comisiones_recursos
    ,division_segura(mprestamos, msaldos_e_inversion) as ratio_comisiones_recursos

from competencia_01

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


## Grupos de variables

In [ ]:
dinero = [
  'mrentabilidad'
  , 'mrentabilidad_annual'
  , 'mcomisiones'
  , 'mactivos_margen'
  , 'mpasivos_margen'
  , 'mcuenta_corriente_adicional'
  , 'mcuenta_corriente'
  , 'mcaja_ahorro'
  , 'mcaja_ahorro_adicional'
  , 'mcaja_ahorro_dolares'
  , 'mcuentas_saldo'
  , 'mautoservicio'
  , 'mtarjeta_visa_consumo'
  , 'mtarjeta_master_consumo'
  , 'mprestamos_personales'
  , 'mprestamos_prendarios'
  , 'mprestamos_hipotecarios'
  , 'mplazo_fijo_dolares'
  , 'mplazo_fijo_pesos'
  , 'minversion1_pesos'
  , 'minversion1_dolares'
  , 'minversion2'
  , 'mpayroll'
  , 'mpayroll2'
  , 'mcuenta_debitos_automaticos'
  , 'mttarjeta_visa_debitos_automaticos'
  , 'mttarjeta_master_debitos_automaticos'
  , 'mpagodeservicios'
  , 'mpagomiscuentas'
  , 'mcajeros_propios_descuentos'
  , 'mtarjeta_visa_descuentos'
  , 'mtarjeta_master_descuentos'
  , 'mcomisiones_mantenimiento'
  , 'mcomisiones_otras'
  , 'mforex_buy'
  , 'mforex_sell'
  , 'mtransferencias_recibidas'
  , 'mtransferencias_emitidas'
  , 'mextraccion_autoservicio'
  , 'mcheques_depositados'
  , 'mcheques_emitidos'
  , 'mcheques_depositados_rechazados'
  , 'mcheques_emitidos_rechazados'
  , 'matm'
  , 'matm_other'
  , 'Master_mfinanciacion_limite'
  , 'Master_msaldototal'
  , 'Master_msaldopesos'
  , 'Master_msaldodolares'
  , 'Master_mconsumospesos'
  , 'Master_mconsumosdolares'
  , 'Master_mlimitecompra'
  , 'Master_madelantopesos'
  , 'Master_madelantodolares'
  , 'Master_mpagado'
  , 'Master_mpagospesos'
  , 'Master_mpagosdolares'
  , 'Master_mconsumototal'
  , 'Master_mpagominimo'
  , 'Visa_mfinanciacion_limite'
  , 'Visa_msaldototal'
  , 'Visa_msaldopesos'
  , 'Visa_msaldodolares'
  , 'Visa_mconsumospesos'
  , 'Visa_mconsumosdolares'
  , 'Visa_mlimitecompra'
  , 'Visa_madelantopesos'
  , 'Visa_madelantodolares'
  , 'Visa_mpagado'
  , 'Visa_mpagospesos'
  , 'Visa_mpagosdolares'
  , 'Visa_mconsumototal'
  , 'Visa_mpagominimo'
  , 'mtarjeta_consumo'
  , 'mprestamos'
  , 'minversion_pesos'
  , 'minversion_dolares'
  , 'mtarjeta_debitos_automaticos'
  , 'mpagos'
  , 'Tarjeta_msaldopesos'
  , 'Tarjeta_msaldodolares'
  , 'Tarjeta_msaldototal'
  , 'Tarjeta_mconsumospesos'
  , 'Tarjeta_mconsumosdolares'
  , 'Tarjeta_mlimitecompra'
  , 'Tarjeta_madelantopesos'
  , 'Tarjeta_madelantodolares'
  , 'Tarjeta_mpagado'
  , 'Tarjeta_mpagospesos'
  , 'Tarjeta_mpagosdolares'
  , 'Tarjeta_mconsumototal'
  , 'Tarjeta_mpagominimo'
]
tileadas=dinero

In [ ]:
%%sql --no-execute
select
    *,
    foto_mes
    , numero_de_cliente
    , cliente_antiguedad
    , row_number() over (partition by numero_de_cliente order by foto_mes) as cliente_antiguedad_2
    , percent_rank() over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_3
    , cume_dist() over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_4
    , ntile(4) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_5

    , ntile(10) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_6
from competencia_01
order by numero_de_cliente, cliente_antiguedad


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,ratio_comisiones_recursos,ratio_comisiones_recursos_1,foto_mes_1,numero_de_cliente_1,cliente_antiguedad_1,cliente_antiguedad_2,cliente_antiguedad_3,cliente_antiguedad_4,cliente_antiguedad_5,cliente_antiguedad_6
0,249221109,202101,1,0,0,61,300,1664.94,17994.40,1605.81,...,0.022461,0.0,202101,249221109,300,1,0.947564,0.947947,4,10
1,249221109,202102,1,0,0,61,301,1002.92,18337.34,1312.91,...,0.012355,0.0,202102,249221109,301,2,0.947831,0.948213,4,10
2,249221109,202103,1,0,0,61,302,769.74,18596.58,1662.56,...,0.024973,0.0,202103,249221109,302,3,0.948291,0.948670,4,10
3,249221109,202104,1,0,0,62,303,1532.79,19935.00,1899.52,...,0.009814,0.0,202104,249221109,303,4,0.948638,0.949016,4,10
4,249221109,202105,1,0,0,62,304,1804.14,20862.15,2693.91,...,0.094253,0.0,202105,249221109,304,5,0.948950,0.949327,4,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981941,1598444941,202106,0,0,0,27,1,0.00,0.00,0.00,...,0.000000,0.0,202106,1598444941,1,1,0.000000,0.001553,1,1
981942,1598471047,202106,1,0,0,28,1,95.20,95.20,0.00,...,0.000000,0.0,202106,1598471047,1,1,0.000000,0.001553,1,1
981943,1598515897,202106,0,0,0,30,1,72.08,72.08,0.00,...,0.000000,0.0,202106,1598515897,1,1,0.000000,0.001553,1,1
981944,1598517059,202106,0,0,0,24,1,0.00,0.00,0.00,...,0.000000,0.0,202106,1598517059,1,1,0.000000,0.001553,1,1


In [ ]:
orden=""
orden=orden+"create or replace competencia_01 as\n"
orden+=f"select *\n"
for campo in tileadas:
  orden+=f",ntile(20) over (partition by foto_mes order by {campo}) as {campo}_ntile\n"
orden+=f"from competencia_01\n"

print (orden)



create or replace competencia_01 as
select *
,ntile(20) over (partition by foto_mes order by mrentabilidad) as mrentabilidad_ntile
,ntile(20) over (partition by foto_mes order by mrentabilidad_annual) as mrentabilidad_annual_ntile
,ntile(20) over (partition by foto_mes order by mcomisiones) as mcomisiones_ntile
,ntile(20) over (partition by foto_mes order by mactivos_margen) as mactivos_margen_ntile
,ntile(20) over (partition by foto_mes order by mpasivos_margen) as mpasivos_margen_ntile
,ntile(20) over (partition by foto_mes order by mcuenta_corriente_adicional) as mcuenta_corriente_adicional_ntile
,ntile(20) over (partition by foto_mes order by mcuenta_corriente) as mcuenta_corriente_ntile
,ntile(20) over (partition by foto_mes order by mcaja_ahorro) as mcaja_ahorro_ntile
,ntile(20) over (partition by foto_mes order by mcaja_ahorro_adicional) as mcaja_ahorro_adicional_ntile
,ntile(20) over (partition by foto_mes order by mcaja_ahorro_dolares) as mcaja_ahorro_dolares_ntile
,ntile(20) 

In [ ]:

%%sql --no-execute
create or replace table competencia_01 as
select *
  ,ntile(20) over (partition by foto_mes order by mrentabilidad) as mrentabilidad_ntile
  ,ntile(20) over (partition by foto_mes order by mrentabilidad_annual) as mrentabilidad_annual_ntile
  ,ntile(20) over (partition by foto_mes order by mcomisiones) as mcomisiones_ntile
  ,ntile(20) over (partition by foto_mes order by mactivos_margen) as mactivos_margen_ntile
  ,ntile(20) over (partition by foto_mes order by mpasivos_margen) as mpasivos_margen_ntile
  ,ntile(20) over (partition by foto_mes order by mcuenta_corriente_adicional) as mcuenta_corriente_adicional_ntile
  ,ntile(20) over (partition by foto_mes order by mcuenta_corriente) as mcuenta_corriente_ntile
  ,ntile(20) over (partition by foto_mes order by mcaja_ahorro) as mcaja_ahorro_ntile
  ,ntile(20) over (partition by foto_mes order by mcaja_ahorro_adicional) as mcaja_ahorro_adicional_ntile
  ,ntile(20) over (partition by foto_mes order by mcaja_ahorro_dolares) as mcaja_ahorro_dolares_ntile
  ,ntile(20) over (partition by foto_mes order by mcuentas_saldo) as mcuentas_saldo_ntile
  ,ntile(20) over (partition by foto_mes order by mautoservicio) as mautoservicio_ntile
  ,ntile(20) over (partition by foto_mes order by mtarjeta_visa_consumo) as mtarjeta_visa_consumo_ntile
  ,ntile(20) over (partition by foto_mes order by mtarjeta_master_consumo) as mtarjeta_master_consumo_ntile
  ,ntile(20) over (partition by foto_mes order by mprestamos_personales) as mprestamos_personales_ntile
  ,ntile(20) over (partition by foto_mes order by mprestamos_prendarios) as mprestamos_prendarios_ntile
  ,ntile(20) over (partition by foto_mes order by mprestamos_hipotecarios) as mprestamos_hipotecarios_ntile
  ,ntile(20) over (partition by foto_mes order by mplazo_fijo_dolares) as mplazo_fijo_dolares_ntile
  ,ntile(20) over (partition by foto_mes order by mplazo_fijo_pesos) as mplazo_fijo_pesos_ntile
  ,ntile(20) over (partition by foto_mes order by minversion1_pesos) as minversion1_pesos_ntile
  ,ntile(20) over (partition by foto_mes order by minversion1_dolares) as minversion1_dolares_ntile
  ,ntile(20) over (partition by foto_mes order by minversion2) as minversion2_ntile
from competencia_01


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [ ]:
%%sql --no-execute
create or replace table competencia_01 as
select *
  ,ntile(20) over (partition by foto_mes order by mpayroll) as mpayroll_ntile
  ,ntile(20) over (partition by foto_mes order by mpayroll2) as mpayroll2_ntile
  ,ntile(20) over (partition by foto_mes order by mcuenta_debitos_automaticos) as mcuenta_debitos_automaticos_ntile
  ,ntile(20) over (partition by foto_mes order by mttarjeta_visa_debitos_automaticos) as mttarjeta_visa_debitos_automaticos_ntile
  ,ntile(20) over (partition by foto_mes order by mttarjeta_master_debitos_automaticos) as mttarjeta_master_debitos_automaticos_ntile
  ,ntile(20) over (partition by foto_mes order by mpagodeservicios) as mpagodeservicios_ntile
  ,ntile(20) over (partition by foto_mes order by mpagomiscuentas) as mpagomiscuentas_ntile
  ,ntile(20) over (partition by foto_mes order by mcajeros_propios_descuentos) as mcajeros_propios_descuentos_ntile
  ,ntile(20) over (partition by foto_mes order by mtarjeta_visa_descuentos) as mtarjeta_visa_descuentos_ntile
  ,ntile(20) over (partition by foto_mes order by mtarjeta_master_descuentos) as mtarjeta_master_descuentos_ntile
  ,ntile(20) over (partition by foto_mes order by mcomisiones_mantenimiento) as mcomisiones_mantenimiento_ntile
  ,ntile(20) over (partition by foto_mes order by mcomisiones_otras) as mcomisiones_otras_ntile
  ,ntile(20) over (partition by foto_mes order by mforex_buy) as mforex_buy_ntile
  ,ntile(20) over (partition by foto_mes order by mforex_sell) as mforex_sell_ntile
  ,ntile(20) over (partition by foto_mes order by mtransferencias_recibidas) as mtransferencias_recibidas_ntile
  ,ntile(20) over (partition by foto_mes order by mtransferencias_emitidas) as mtransferencias_emitidas_ntile
  ,ntile(20) over (partition by foto_mes order by mextraccion_autoservicio) as mextraccion_autoservicio_ntile
  ,ntile(20) over (partition by foto_mes order by mcheques_depositados) as mcheques_depositados_ntile
  ,ntile(20) over (partition by foto_mes order by mcheques_emitidos) as mcheques_emitidos_ntile
from competencia_01


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [ ]:
%%sql --no-execute
create or replace table competencia_01 as
select *
  ,ntile(20) over (partition by foto_mes order by mcheques_depositados_rechazados) as mcheques_depositados_rechazados_ntile
  ,ntile(20) over (partition by foto_mes order by mcheques_emitidos_rechazados) as mcheques_emitidos_rechazados_ntile
  ,ntile(20) over (partition by foto_mes order by matm) as matm_ntile
  ,ntile(20) over (partition by foto_mes order by matm_other) as matm_other_ntile
  ,ntile(20) over (partition by foto_mes order by Master_mfinanciacion_limite) as Master_mfinanciacion_limite_ntile
  ,ntile(20) over (partition by foto_mes order by Master_msaldototal) as Master_msaldototal_ntile
  ,ntile(20) over (partition by foto_mes order by Master_msaldopesos) as Master_msaldopesos_ntile
  ,ntile(20) over (partition by foto_mes order by Master_msaldodolares) as Master_msaldodolares_ntile
  ,ntile(20) over (partition by foto_mes order by Master_mconsumospesos) as Master_mconsumospesos_ntile
  ,ntile(20) over (partition by foto_mes order by Master_mconsumosdolares) as Master_mconsumosdolares_ntile
  ,ntile(20) over (partition by foto_mes order by Master_mlimitecompra) as Master_mlimitecompra_ntile
  ,ntile(20) over (partition by foto_mes order by Master_madelantopesos) as Master_madelantopesos_ntile
  ,ntile(20) over (partition by foto_mes order by Master_madelantodolares) as Master_madelantodolares_ntile
  ,ntile(20) over (partition by foto_mes order by Master_mpagado) as Master_mpagado_ntile
  ,ntile(20) over (partition by foto_mes order by Master_mpagospesos) as Master_mpagospesos_ntile
  ,ntile(20) over (partition by foto_mes order by Master_mpagosdolares) as Master_mpagosdolares_ntile
  ,ntile(20) over (partition by foto_mes order by Master_mconsumototal) as Master_mconsumototal_ntile
  ,ntile(20) over (partition by foto_mes order by Master_mpagominimo) as Master_mpagominimo_ntile
  ,ntile(20) over (partition by foto_mes order by Visa_mfinanciacion_limite) as Visa_mfinanciacion_limite_ntile
  ,ntile(20) over (partition by foto_mes order by Visa_msaldototal) as Visa_msaldototal_ntile
  ,ntile(20) over (partition by foto_mes order by Visa_msaldopesos) as Visa_msaldopesos_ntile
from competencia_01


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [ ]:
%%sql
create or replace table competencia_01 as
select *
  ,ntile(20) over (partition by foto_mes order by Visa_msaldodolares) as Visa_msaldodolares_ntile
  ,ntile(20) over (partition by foto_mes order by Visa_mconsumospesos) as Visa_mconsumospesos_ntile
  ,ntile(20) over (partition by foto_mes order by Visa_mconsumosdolares) as Visa_mconsumosdolares_ntile
  ,ntile(20) over (partition by foto_mes order by Visa_mlimitecompra) as Visa_mlimitecompra_ntile
  ,ntile(20) over (partition by foto_mes order by Visa_madelantopesos) as Visa_madelantopesos_ntile
  ,ntile(20) over (partition by foto_mes order by Visa_madelantodolares) as Visa_madelantodolares_ntile
  ,ntile(20) over (partition by foto_mes order by Visa_mpagado) as Visa_mpagado_ntile
  ,ntile(20) over (partition by foto_mes order by Visa_mpagospesos) as Visa_mpagospesos_ntile
  ,ntile(20) over (partition by foto_mes order by Visa_mpagosdolares) as Visa_mpagosdolares_ntile
  ,ntile(20) over (partition by foto_mes order by Visa_mconsumototal) as Visa_mconsumototal_ntile
  ,ntile(20) over (partition by foto_mes order by Visa_mpagominimo) as Visa_mpagominimo_ntile
  ,ntile(20) over (partition by foto_mes order by mtarjeta_consumo) as mtarjeta_consumo_ntile
  ,ntile(20) over (partition by foto_mes order by mprestamos) as mprestamos_ntile
  ,ntile(20) over (partition by foto_mes order by minversion_pesos) as minversion_pesos_ntile
  ,ntile(20) over (partition by foto_mes order by minversion_dolares) as minversion_dolares_ntile
  ,ntile(20) over (partition by foto_mes order by mtarjeta_debitos_automaticos) as mtarjeta_debitos_automaticos_ntile
  ,ntile(20) over (partition by foto_mes order by mpagos) as mpagos_ntile
  ,ntile(20) over (partition by foto_mes order by Tarjeta_msaldopesos) as Tarjeta_msaldopesos_ntile
  ,ntile(20) over (partition by foto_mes order by Tarjeta_msaldodolares) as Tarjeta_msaldodolares_ntile
  ,ntile(20) over (partition by foto_mes order by Tarjeta_msaldototal) as Tarjeta_msaldototal_ntile
  ,ntile(20) over (partition by foto_mes order by Tarjeta_mconsumospesos) as Tarjeta_mconsumospesos_ntile
  ,ntile(20) over (partition by foto_mes order by Tarjeta_mconsumosdolares) as Tarjeta_mconsumosdolares_ntile
  ,ntile(20) over (partition by foto_mes order by Tarjeta_mlimitecompra) as Tarjeta_mlimitecompra_ntile
  ,ntile(20) over (partition by foto_mes order by Tarjeta_madelantopesos) as Tarjeta_madelantopesos_ntile
  ,ntile(20) over (partition by foto_mes order by Tarjeta_madelantodolares) as Tarjeta_madelantodolares_ntile
  ,ntile(20) over (partition by foto_mes order by Tarjeta_mpagado) as Tarjeta_mpagado_ntile
  ,ntile(20) over (partition by foto_mes order by Tarjeta_mpagospesos) as Tarjeta_mpagospesos_ntile
  ,ntile(20) over (partition by foto_mes order by Tarjeta_mpagosdolares) as Tarjeta_mpagosdolares_ntile
  ,ntile(20) over (partition by foto_mes order by Tarjeta_mconsumototal) as Tarjeta_mconsumototal_ntile
  ,ntile(20) over (partition by foto_mes order by Tarjeta_mpagominimo) as Tarjeta_mpagominimo_ntile
from competencia_01

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [ ]:
stop
grabar_aumentada()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
%%sql --no-execute
create or replace table competencia_01 as
select
  *
  , lag(mpayroll, 1) over (partition by numero_de_cliente order by foto_mes) as mpayroll_lag1
  , lag(mpayroll, 2) over (partition by numero_de_cliente order by foto_mes) as mpayroll_lag2
  , lag(mpayroll, 3) over (partition by numero_de_cliente order by foto_mes) as mpayroll_lag3
from competencia_01



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


## Otros agregados:
promedios de valores en meses
cambios en cant de tarjetas u otros.
comparar un valor con el mínimo, máximo o promedio de los últimos meses,


Podemos calcular el delta (diferencia) entre el valor pasado y el presente, para uno o varios meses


In [ ]:
%%sql --no-execute
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  , ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
  , ctrx_quarter - lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
from competencia_01
limit 10


Skipping execution...

Si necesitamos ya no solo traer un valor del pasado, sino una secuencia de valores, por ejemplo para calcular la media móvil con los últimos 3 meses anteriores? se puede hacer fácilmente


In [ ]:
%%sql --no-execute
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  , lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
  , lag(ctrx_quarter, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctrx_quarter
  , avg(ctrx_quarter) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 3 preceding and current row) as avg_3_ctrx_quarter
from competencia_01
order by numero_de_cliente, foto_mes desc
limit 10


Skipping execution...

Si embargo puede resultar incómodo escribir constantemente el over partition sobre todo si se buscan aplicar muchas veces para distintas funciones. Para reducir el código se puede usar la siguiente sintaxis



In [ ]:
%%sql --no-execute
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , avg(ctrx_quarter) over ventana_3 as ctrx_quarter_media_3
  , max(ctrx_quarter) over ventana_3 as ctrx_quarter_max_3
  , min(ctrx_quarter) over ventana_3 as ctrx_quarter_min_3
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 2 preceding and current row)
limit 10


Skipping execution...

Para saber más que funciones tenemos disponibles, recomiendo ver los siguientes links:

https://duckdb.org/docs/archive/0.8.1/sql/window_functions
https://duckdb.org/docs/archive/0.8.1/sql/aggregates
Un caso más, que ni me voy a molestar en explicar que significa...


In [ ]:
ver_slope=[
  'mrentabilidad'
, 'cproductos'
, 'mpayroll'
, 'mpayroll2'
, 'cpayroll'
, 'mcuentas_saldo'
, 'ccuenta_debitos_automaticos'
, 'ccajas_transacciones'
, 'ccajas_consultas'
, 'ccajas_depositos'
, 'ccajas_extracciones'
, 'Master_status'
, 'Visa_status'
, 'ctarjeta'
, 'ctarjeta_transacciones'
, 'cprestamos'
, 'cinversion'
, 'cseguro'
, 'ctarjeta_debitos_automaticos'
, 'mcuentas_saldo'
, 'msaldos_e_inversion'
]


In [ ]:
orden=""
orden+=f"select *\n"
for campo in ver_slope:
  orden+=f",regr_slope({campo}, foto_mes) over ventana_3 as {campo}_slope_3\n"
orden+=f"from competencia_01\n"
orden+=f"window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)\n"


print (orden)



select *
,regr_slope(mrentabilidad, foto_mes) over ventana_3 as mrentabilidad_slope_3
,regr_slope(cproductos, foto_mes) over ventana_3 as cproductos_slope_3
,regr_slope(mpayroll, foto_mes) over ventana_3 as mpayroll_slope_3
,regr_slope(mpayroll2, foto_mes) over ventana_3 as mpayroll2_slope_3
,regr_slope(cpayroll, foto_mes) over ventana_3 as cpayroll_slope_3
,regr_slope(mcuentas_saldo, foto_mes) over ventana_3 as mcuentas_saldo_slope_3
,regr_slope(ccuenta_debitos_automaticos, foto_mes) over ventana_3 as ccuenta_debitos_automaticos_slope_3
,regr_slope(ccajas_transacciones, foto_mes) over ventana_3 as ccajas_transacciones_slope_3
,regr_slope(ccajas_consultas, foto_mes) over ventana_3 as ccajas_consultas_slope_3
,regr_slope(ccajas_depositos, foto_mes) over ventana_3 as ccajas_depositos_slope_3
,regr_slope(ccajas_extracciones, foto_mes) over ventana_3 as ccajas_extracciones_slope_3
,regr_slope(Master_status, foto_mes) over ventana_3 as Master_status_slope_3
,regr_slope(Visa_status, foto_mes

In [ ]:
%%sql --no-execute
create or replace table competencia_01 as
select *
  ,regr_slope(mrentabilidad, foto_mes) over ventana_3 as mrentabilidad_slope_3
  ,regr_slope(cproductos, foto_mes) over ventana_3 as cproductos_slope_3
  ,regr_slope(mpayroll, foto_mes) over ventana_3 as mpayroll_slope_3
  ,regr_slope(mpayroll2, foto_mes) over ventana_3 as mpayroll2_slope_3
  ,regr_slope(ccuenta_debitos_automaticos, foto_mes) over ventana_3 as ccuenta_debitos_automaticos_slope_3
  ,regr_slope(ccajas_transacciones, foto_mes) over ventana_3 as ccajas_transacciones_slope_3
  ,regr_slope(ccajas_consultas, foto_mes) over ventana_3 as ccajas_consultas_slope_3
  ,regr_slope(ccajas_depositos, foto_mes) over ventana_3 as ccajas_depositos_slope_3
  ,regr_slope(ccajas_extracciones, foto_mes) over ventana_3 as ccajas_extracciones_slope_3
  ,regr_slope(Master_status, foto_mes) over ventana_3 as Master_status_slope_3
  ,regr_slope(Visa_status, foto_mes) over ventana_3 as Visa_status_slope_3
  ,regr_slope(ctarjeta, foto_mes) over ventana_3 as ctarjeta_slope_3
  ,regr_slope(ctarjeta_transacciones, foto_mes) over ventana_3 as ctarjeta_transacciones_slope_3
  ,regr_slope(cprestamos, foto_mes) over ventana_3 as cprestamos_slope_3
  ,regr_slope(cinversion, foto_mes) over ventana_3 as cinversion_slope_3
  ,regr_slope(cseguro, foto_mes) over ventana_3 as cseguro_slope_3
  ,regr_slope(ctarjeta_debitos_automaticos, foto_mes) over ventana_3 as ctarjeta_debitos_automaticos_slope_3
  ,regr_slope(mcuentas_saldo, foto_mes) over ventana_3 as mcuentas_saldo_slope_3
  ,regr_slope(msaldos_e_inversion, foto_mes) over ventana_3 as msaldos_e_inversion_slope_3

from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 2 preceding and current row)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [ ]:
%%sql --no-execute
create or replace table competencia_01 as
select *
  ,regr_slope(mrentabilidad, foto_mes) over ventana_4 as mrentabilidad_slope_4
  ,regr_slope(cproductos, foto_mes) over ventana_4 as cproductos_slope_4
  ,regr_slope(mpayroll, foto_mes) over ventana_4 as mpayroll_slope_4
  ,regr_slope(mpayroll2, foto_mes) over ventana_4 as mpayroll2_slope_4
  ,regr_slope(ccuenta_debitos_automaticos, foto_mes) over ventana_4 as ccuenta_debitos_automaticos_slope_4
  ,regr_slope(ccajas_transacciones, foto_mes) over ventana_4 as ccajas_transacciones_slope_4
  ,regr_slope(ccajas_consultas, foto_mes) over ventana_4 as ccajas_consultas_slope_4
  ,regr_slope(ccajas_depositos, foto_mes) over ventana_4 as ccajas_depositos_slope_4
  ,regr_slope(ccajas_extracciones, foto_mes) over ventana_4 as ccajas_extracciones_slope_4
  ,regr_slope(Master_status, foto_mes) over ventana_4 as Master_status_slope_4
  ,regr_slope(Visa_status, foto_mes) over ventana_4 as Visa_status_slope_4
  ,regr_slope(ctarjeta, foto_mes) over ventana_4 as ctarjeta_slope_4
  ,regr_slope(ctarjeta_transacciones, foto_mes) over ventana_4 as ctarjeta_transacciones_slope_4
  ,regr_slope(cprestamos, foto_mes) over ventana_4 as cprestamos_slope_4
  ,regr_slope(cinversion, foto_mes) over ventana_4 as cinversion_slope_4
  ,regr_slope(cseguro, foto_mes) over ventana_4 as cseguro_slope_4
  ,regr_slope(ctarjeta_debitos_automaticos, foto_mes) over ventana_4 as ctarjeta_debitos_automaticos_slope_4
  ,regr_slope(mcuentas_saldo, foto_mes) over ventana_4 as mcuentas_saldo_slope_4
  ,regr_slope(msaldos_e_inversion, foto_mes) over ventana_4 as msaldos_e_inversion_slope_4

from competencia_01
window ventana_4 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [ ]:
grabar_aumentada()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
%%sql --no-execute
select
  numero_de_cliente
  , clase_ternaria
  , foto_mes
  , ctrx_quarter
  ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_3 as ctrx_quarter_slope_3
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 2 preceding and current row)
limit 100


Skipping execution...

In [ ]:
# Eliminar las columnas que fueron a ntile y a suma
orden=""

for campo in tileadas:
  orden=orden+"ALTER TABLE competencia_01 \n"
  orden+=f" DROP COLUMN {campo};\n"

print (orden)

ALTER TABLE competencia_01 
 DROP COLUMN mrentabilidad;
ALTER TABLE competencia_01 
 DROP COLUMN mrentabilidad_annual;
ALTER TABLE competencia_01 
 DROP COLUMN mcomisiones;
ALTER TABLE competencia_01 
 DROP COLUMN mactivos_margen;
ALTER TABLE competencia_01 
 DROP COLUMN mpasivos_margen;
ALTER TABLE competencia_01 
 DROP COLUMN mcuenta_corriente_adicional;
ALTER TABLE competencia_01 
 DROP COLUMN mcuenta_corriente;
ALTER TABLE competencia_01 
 DROP COLUMN mcaja_ahorro;
ALTER TABLE competencia_01 
 DROP COLUMN mcaja_ahorro_adicional;
ALTER TABLE competencia_01 
 DROP COLUMN mcaja_ahorro_dolares;
ALTER TABLE competencia_01 
 DROP COLUMN mcuentas_saldo;
ALTER TABLE competencia_01 
 DROP COLUMN mautoservicio;
ALTER TABLE competencia_01 
 DROP COLUMN mtarjeta_visa_consumo;
ALTER TABLE competencia_01 
 DROP COLUMN mtarjeta_master_consumo;
ALTER TABLE competencia_01 
 DROP COLUMN mprestamos_personales;
ALTER TABLE competencia_01 
 DROP COLUMN mprestamos_prendarios;
ALTER TABLE competencia_01 

In [ ]:
%%sql --no-execute
ALTER TABLE competencia_01
 DROP COLUMN mrentabilidad;
ALTER TABLE competencia_01
 DROP COLUMN mrentabilidad_annual;
ALTER TABLE competencia_01
 DROP COLUMN mcomisiones;
ALTER TABLE competencia_01
 DROP COLUMN mactivos_margen;
ALTER TABLE competencia_01
 DROP COLUMN mpasivos_margen;
ALTER TABLE competencia_01
 DROP COLUMN mcuenta_corriente_adicional;
ALTER TABLE competencia_01
 DROP COLUMN mcuenta_corriente;
ALTER TABLE competencia_01
 DROP COLUMN mcaja_ahorro;
ALTER TABLE competencia_01
 DROP COLUMN mcaja_ahorro_adicional;
ALTER TABLE competencia_01
 DROP COLUMN mcaja_ahorro_dolares;
ALTER TABLE competencia_01
 DROP COLUMN mcuentas_saldo;
ALTER TABLE competencia_01
 DROP COLUMN mautoservicio;
ALTER TABLE competencia_01
 DROP COLUMN mtarjeta_visa_consumo;
ALTER TABLE competencia_01
 DROP COLUMN mtarjeta_master_consumo;
ALTER TABLE competencia_01
 DROP COLUMN mprestamos_personales;
ALTER TABLE competencia_01
 DROP COLUMN mprestamos_prendarios;
ALTER TABLE competencia_01
 DROP COLUMN mprestamos_hipotecarios;
ALTER TABLE competencia_01
 DROP COLUMN mplazo_fijo_dolares;
ALTER TABLE competencia_01
 DROP COLUMN mplazo_fijo_pesos;
ALTER TABLE competencia_01
 DROP COLUMN minversion1_pesos;
ALTER TABLE competencia_01
 DROP COLUMN minversion1_dolares;
ALTER TABLE competencia_01
 DROP COLUMN minversion2;
ALTER TABLE competencia_01
 DROP COLUMN mpayroll;
ALTER TABLE competencia_01
 DROP COLUMN mpayroll2;
ALTER TABLE competencia_01
 DROP COLUMN mcuenta_debitos_automaticos;
ALTER TABLE competencia_01
 DROP COLUMN mttarjeta_visa_debitos_automaticos;
ALTER TABLE competencia_01
 DROP COLUMN mttarjeta_master_debitos_automaticos;
ALTER TABLE competencia_01
 DROP COLUMN mpagodeservicios;
ALTER TABLE competencia_01
 DROP COLUMN mpagomiscuentas;
ALTER TABLE competencia_01
 DROP COLUMN mcajeros_propios_descuentos;
ALTER TABLE competencia_01
 DROP COLUMN mtarjeta_visa_descuentos;
ALTER TABLE competencia_01
 DROP COLUMN mtarjeta_master_descuentos;
ALTER TABLE competencia_01
 DROP COLUMN mcomisiones_mantenimiento;
ALTER TABLE competencia_01
 DROP COLUMN mcomisiones_otras;
ALTER TABLE competencia_01
 DROP COLUMN mforex_buy;
ALTER TABLE competencia_01
 DROP COLUMN mforex_sell;
ALTER TABLE competencia_01
 DROP COLUMN mtransferencias_recibidas;
ALTER TABLE competencia_01
 DROP COLUMN mtransferencias_emitidas;
ALTER TABLE competencia_01
 DROP COLUMN mextraccion_autoservicio;
ALTER TABLE competencia_01
 DROP COLUMN mcheques_depositados;
ALTER TABLE competencia_01
 DROP COLUMN mcheques_emitidos;
ALTER TABLE competencia_01
 DROP COLUMN mcheques_depositados_rechazados;
ALTER TABLE competencia_01
 DROP COLUMN mcheques_emitidos_rechazados;
ALTER TABLE competencia_01
 DROP COLUMN matm;
ALTER TABLE competencia_01
 DROP COLUMN matm_other;
ALTER TABLE competencia_01
 DROP COLUMN Master_mfinanciacion_limite;
ALTER TABLE competencia_01
 DROP COLUMN Master_msaldototal;
ALTER TABLE competencia_01
 DROP COLUMN Master_msaldopesos;
ALTER TABLE competencia_01
 DROP COLUMN Master_msaldodolares;
ALTER TABLE competencia_01
 DROP COLUMN Master_mconsumospesos;
ALTER TABLE competencia_01
 DROP COLUMN Master_mconsumosdolares;
ALTER TABLE competencia_01
 DROP COLUMN Master_mlimitecompra;
ALTER TABLE competencia_01
 DROP COLUMN Master_madelantopesos;
ALTER TABLE competencia_01
 DROP COLUMN Master_madelantodolares;
ALTER TABLE competencia_01
 DROP COLUMN Master_mpagado;
ALTER TABLE competencia_01
 DROP COLUMN Master_mpagospesos;
ALTER TABLE competencia_01
 DROP COLUMN Master_mpagosdolares;
ALTER TABLE competencia_01
 DROP COLUMN Master_mconsumototal;
ALTER TABLE competencia_01
 DROP COLUMN Master_mpagominimo;
ALTER TABLE competencia_01
 DROP COLUMN Visa_mfinanciacion_limite;
ALTER TABLE competencia_01
 DROP COLUMN Visa_msaldototal;
ALTER TABLE competencia_01
 DROP COLUMN Visa_msaldopesos;
ALTER TABLE competencia_01
 DROP COLUMN Visa_msaldodolares;
ALTER TABLE competencia_01
 DROP COLUMN Visa_mconsumospesos;
ALTER TABLE competencia_01
 DROP COLUMN Visa_mconsumosdolares;
ALTER TABLE competencia_01
 DROP COLUMN Visa_mlimitecompra;
ALTER TABLE competencia_01
 DROP COLUMN Visa_madelantopesos;
ALTER TABLE competencia_01
 DROP COLUMN Visa_madelantodolares;
ALTER TABLE competencia_01
 DROP COLUMN Visa_mpagado;
ALTER TABLE competencia_01
 DROP COLUMN Visa_mpagospesos;
ALTER TABLE competencia_01
 DROP COLUMN Visa_mpagosdolares;
ALTER TABLE competencia_01
 DROP COLUMN Visa_mconsumototal;
ALTER TABLE competencia_01
 DROP COLUMN Visa_mpagominimo;
ALTER TABLE competencia_01
 DROP COLUMN mtarjeta_consumo;
ALTER TABLE competencia_01
 DROP COLUMN mprestamos;
ALTER TABLE competencia_01
 DROP COLUMN minversion_pesos;
ALTER TABLE competencia_01
 DROP COLUMN minversion_dolares;
ALTER TABLE competencia_01
 DROP COLUMN mtarjeta_debitos_automaticos;
ALTER TABLE competencia_01
 DROP COLUMN mpagos;
ALTER TABLE competencia_01
 DROP COLUMN Tarjeta_msaldopesos;
ALTER TABLE competencia_01
 DROP COLUMN Tarjeta_msaldodolares;
ALTER TABLE competencia_01
 DROP COLUMN Tarjeta_msaldototal;
ALTER TABLE competencia_01
 DROP COLUMN Tarjeta_mconsumospesos;
ALTER TABLE competencia_01
 DROP COLUMN Tarjeta_mconsumosdolares;
ALTER TABLE competencia_01
 DROP COLUMN Tarjeta_mlimitecompra;
ALTER TABLE competencia_01
 DROP COLUMN Tarjeta_madelantopesos;
ALTER TABLE competencia_01
 DROP COLUMN Tarjeta_madelantodolares;
ALTER TABLE competencia_01
 DROP COLUMN Tarjeta_mpagado;
ALTER TABLE competencia_01
 DROP COLUMN Tarjeta_mpagospesos;
ALTER TABLE competencia_01
 DROP COLUMN Tarjeta_mpagosdolares;
ALTER TABLE competencia_01
 DROP COLUMN Tarjeta_mconsumototal;
ALTER TABLE competencia_01
 DROP COLUMN Tarjeta_mpagominimo;


,Success


In [ ]:
# grabar_reducida()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Se grabó /Data Science y similares/Maestría Data Mining Exactas/dmeyf/dmeyf2024/datasets/competencia_01_reducida.csv
